#**Enhanced Pedestrian Intention Detection for ADAS and Autonomous Vehicles**
Demo of the PID model on the JAAD dataset.


##Step 1: Mount google drive and create folder for output

In [ ]:
from google.colab import drive

# Check if Google Drive is mounted
if not drive.is_mounted():
    # Mount Google Drive if not already mounted
    drive.mount('/content/drive')

# Get the path to the desired parent folder (replace with your desired path)
parent_folder_path = '/content/drive/My Drive'

# Create the folder
folder_name = 'pid_model_output'
folder_path = os.path.join(parent_folder_path, folder_name)

# Check if the folder already exists
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Print the path to the created folder
print(f"Folder created at: {folder_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Steps 2-5
## Step 2: Clone the JAAD repositry to the content directory on google colab:
https://github.com/ykotseruba/JAAD

##Step 3: Upload a zipped file of the JAAD clips into your google drive:
http://data.nvision2.eecs.yorku.ca/JAAD_dataset/data/JAAD_clips.zip

##Step 4: Unzip JAAD_clips.zip and copy the jaad_data.py file into the content directory on google colab.

##Step 5: Clone the YOLOP repository into the content directory.

In [ ]:
!unzip "/content/drive/My Drive/JAAD/JAAD_clips.zip" -d "/content"
!git clone https://github.com/ykotseruba/JAAD.git
!cp '/content/JAAD/jaad_data.py' '/content'
!git clone https://github.com/hustvl/YOLOP

Archive:  /content/drive/My Drive/JAAD/JAAD_clips.zip
replace /content/JAAD_clips/video_0001.mp4? [y]es, [n]o, [A]ll, [N]one, [r]ename: Cloning into 'JAAD'...
remote: Enumerating objects: 6149, done.
remote: Counting objects: 100% (718/718), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 6149 (delta 668), reused 695 (delta 652), pack-reused 5431
Receiving objects: 100% (6149/6149), 42.16 MiB | 12.21 MiB/s, done.
Resolving deltas: 100% (5487/5487), done.
fatal: destination path 'YOLOP' already exists and is not an empty directory.


## Step 6: Import OpenCV and Tensorflow

In [ ]:
import csv
import os
import pandas as pd
import h5py
import numpy as np
import cv2

import tensorflow as tf

from jaad_data import JAAD

## Step 6: Define a function YOLOPdetect that returns the drivable area and lane segmentation for an image.

 This is a modified version of the YOLOPdetect function that is in demo.py in the YOLOP directory. All credit goes to its authors.

In [ ]:
import argparse
import os, sys

BASE_DIR = os.path.dirname('/content/YOLOP')
sys.path.append('/content/YOLOP')

import shutil
import time
from pathlib import Path
import imageio

print(sys.path)
import torch
import torch.backends.cudnn as cudnn
from numpy import random
import scipy.special
import torchvision.transforms as transforms
import PIL.Image as image

!sudo pip install yacs
from lib.config import cfg
from lib.config import update_config
from lib.utils.utils import create_logger, select_device, time_synchronized
from lib.models import get_net
from lib.dataset import LoadImages, LoadStreams
from lib.core.general import non_max_suppression, scale_coords
from lib.utils import plot_one_box,show_seg_result
from lib.core.function import AverageMeter
from lib.core.postprocess import morphological_process, connect_lane
from lib.utils import letterbox_for_img, clean_str
from tqdm import tqdm
from numba import cuda

from google.colab.patches import cv2_imshow

normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    )

transform=transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])

class opt:
      device = '0'
      weights = '/content/YOLOP/weights/End-to-end.pth'
      source = '/content/YOLOPstream.png'
      img_size = 640
      conf_thres = 0.25
      iou_thres = 0.45
      save_dir = '/content/YOLOP/output'

opt = opt()

device = select_device(None,opt.device)
half = device.type != 'cpu'  # half precision only supported on CUDA

# Load model
yolop_model = get_net(cfg)
checkpoint = torch.load(opt.weights, map_location= device)
yolop_model.load_state_dict(checkpoint['state_dict'])
yolop_model = yolop_model.to(device)
if half:
    yolop_model.half()  # to FP16

img = torch.zeros((1, 3, opt.img_size, opt.img_size), device=device)  # init img
_ = yolop_model(img.half() if half else img) if device.type != 'cpu' else None  # run once
yolop_model.eval()

def YOLOPdetect(img0):

    h0, w0 = img0.shape[:2]
    img, ratio, pad = letterbox_for_img(img0, new_shape=opt.img_size, auto=True)
    h, w = img.shape[:2]
    shapes = (h0, w0), ((h / h0, w / w0), pad)

    dataset = [(None, img, img0, None, shapes)]
    bs = 1  # batch_size


    # Get names and colors
    names = yolop_model.module.names if hasattr(yolop_model, 'module') else yolop_model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]


    # Run inference
    t0 = time.time()

    inf_time = AverageMeter()
    nms_time = AverageMeter()



    da_seg_mask = None

    for i, (path, img, img_det, vid_cap,shapes) in enumerate(dataset):
        with torch.no_grad():
          torch.cuda.empty_cache()
          img = transform(img).to(device)
          img = img.half() if half else img.float()  # uint8 to fp16/32
          if img.ndimension() == 3:
              img = img.unsqueeze(0)
          # Inference
          t1 = time_synchronized()
          det_out, da_seg_out,ll_seg_out= yolop_model(img)
          t2 = time_synchronized()
          # if i == 0:
          #     print(det_out)
          inf_out, _ = det_out
          inf_time.update(t2-t1,img.size(0))

          # Apply NMS
          t3 = time_synchronized()
          det_pred = non_max_suppression(inf_out, conf_thres=opt.conf_thres, iou_thres=opt.iou_thres, classes=None, agnostic=False)
          t4 = time_synchronized()

          nms_time.update(t4-t3,img.size(0))
          det=det_pred[0]

          _, _, height, width = img.shape
          h,w,_=img_det.shape
          pad_w, pad_h = shapes[1][1]
          pad_w = int(pad_w)
          pad_h = int(pad_h)
          ratio = shapes[1][0][1]

          da_predict = da_seg_out[:, :, pad_h:(height-pad_h),pad_w:(width-pad_w)]
          da_seg_mask = torch.nn.functional.interpolate(da_predict, scale_factor=int(1/ratio), mode='bilinear')
          _, da_seg_mask = torch.max(da_seg_mask, 1)
          da_seg_mask = da_seg_mask.int().squeeze().cpu().numpy()

          ll_predict = ll_seg_out[:, :,pad_h:(height-pad_h),pad_w:(width-pad_w)]
          ll_seg_mask = torch.nn.functional.interpolate(ll_predict, scale_factor=int(1/ratio), mode='bilinear')
          _, ll_seg_mask = torch.max(ll_seg_mask, 1)
          ll_seg_mask = ll_seg_mask.int().squeeze().cpu().numpy()

          img_det = show_seg_result(img_det, (da_seg_mask, ll_seg_mask), _, _, is_demo=True)

          del ll_predict
          del da_predict
          del ll_seg_mask
          del da_seg_mask
          del inf_out
          del det_out
          del det_pred
          del img
          torch.cuda.empty_cache()
    img_det = cv2.resize(img_det, (w0,h0))
    return img_det

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/YOLOP', '/content/YOLOP']


## Steps 7-8

## Step 7: Upload the two files for the pre-trained model found on the github repo 'desenet_model.json' & 'densenet_weights.h5' to google drive

## Step 8: Setup YOLO-NAS-Pose through super-gradients, Deep OC-Sort through boxmot, and the DenseNet model through tensorflow

In [ ]:
!pip install super-gradients
from super_gradients.common.object_names import Models
from super_gradients.training import models
from super_gradients.training.utils.visualization.pose_estimation import draw_skeleton # draws skeletal graph given pose keypoints

# yolo nas pose
yolo_model = models.get(Models.YOLO_NAS_POSE_S, pretrained_weights="coco_pose")

yolo_model = yolo_model.to("cuda")

!pip install boxmot
from boxmot import DeepOCSORT

# deep oc-sort
tracker = DeepOCSORT(
    model_weights=Path('osnet_x0_25_msmt17.pt'), # which ReID model to use
    device='cuda:0',
    fp16=True,
    det_thresh=0.2
)

# densenet
with open('/content/drive/My Drive/densenet_model.json', 'r') as json_file:
    json_savedModel= json_file.read()

model_j = tf.keras.models.model_from_json(json_savedModel)
model_j.load_weights('/content/drive/My Drive/densenet_weights.h5')

def pred_func(X_test):
  predictions = model_j.predict(X_test[0:1], verbose=0)
  Y = np.argmax(predictions[0], axis=0)

  return Y

[2024-05-10 16:36:20] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


[2024-05-10 16:36:22] WARNING - checkpoint_utils.py - :warning: The pre-trained models provided by SuperGradients may have their own licenses or terms and conditions derived from the dataset used for pre-training.
 It is your responsibility to determine whether you have permission to use the models for your use case.
 The model you have requested was pre-trained on the coco_pose dataset, published under the following terms: https://cocodataset.org/#termsofuse
[2024-05-10 16:36:22] INFO - checkpoint_utils.py - License Notification: YOLO-NAS-POSE pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS-POSE.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2024-05-10 16:36:22] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_pose_s


2024-05-10 16:36:29.291 | INFO     | boxmot.utils.torch_utils:select_device:52 - Yolo Tracking v10.1.67 🚀 Python-3.10.12 torch-2.2.1+cu121
CUDA:0 (NVIDIA L4, 22700MiB)
Downloading...
From: https://drive.google.com/uc?id=1sSwXSUlj4_tHZequ_iZ8w_Jh0VaRQMqF
To: /content/osnet_x0_25_msmt17.pt
100%|██████████| 3.06M/3.06M [00:00<00:00, 24.2MB/s]
2024-05-10 16:36:31.109 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"


## Run to demo the model

Video files will be saved to google drive (make sure the folder exists)

In [ ]:
jaad = JAAD(data_path='/content/JAAD')

for video_id in range(1,31):

    # Capture JAAD clip from video ID
    video_id_two_digit = f"0{video_id}" if video_id < 10 else f"{video_id}"
    video_name="video_00" + video_id_two_digit + ".mp4"
    vid = cv2.VideoCapture(f"/content/JAAD_clips/{video_name}")
    ret, img = vid.read()

    # Initialize a variable to store the time of the previous frame.
    time1 = 0
    frame = 0;

    # Initialize arrays
    img_array = []
    rolling_data = {}

    # Loop through every frame in the video
    while vid.isOpened():
        ret, img = vid.read()

        if not ret:
            break

        h, w, _ = img.shape

        # Detect pose keypoints and bounding boxes
        media_predictions = yolo_model.predict(img, conf=0.2, skip_image_resizing=True)
        poses= media_predictions.prediction.poses
        bboxes= media_predictions.prediction.bboxes_xyxy
        conf = media_predictions.prediction.scores
        edge_links= media_predictions.prediction.edge_links
        edge_colors=media_predictions.prediction.edge_colors
        keypoint_colors=media_predictions.prediction.keypoint_colors

        # Construct skeletal graph
        for i in range(len(poses)):
            img = draw_skeleton(img,poses[i],conf[i],edge_links,edge_colors,2,keypoint_colors,4,0.0,False)

        # Add bounding boxes to array dets for Deep OC-Sort input
        dets = np.empty((0,6))
        for i, bbox in enumerate(bboxes):
            det = np.concatenate((bbox,[conf[i]],[0]))
            dets = np.append(dets, [det], axis=0)

        if(dets.size!=0):
            # Run Deep OC-Sort (update tracker)
            tracks = tracker.update(dets, img) # --> M X (x, y, x, y, id, conf, cls, ind)

            # Highlight drivable area and lane segmentation onto frame
            img = YOLOPdetect(img)

            if(tracks.size!=0):
                xyxys = tracks[:, 0:4].astype('int') # float64 to int
                ids = tracks[:, 4].astype('int') # float64 to int
                confs = tracks[:, 5]
                clss = tracks[:, 6].astype('int') # float64 to int
                inds = tracks[:, 7].astype('int') # float64 to int

                # Loop over every bounding box
                for i, bbox in enumerate(xyxys):

                    # Expand bounding box
                    x1 = bbox[0] - 10 if bbox[0]  - 10 > 0 else 0
                    y1 = bbox[1] - 10 if bbox[1]  - 10 > 0 else 0
                    x2 = bbox[2] + 10 if bbox[2]  + 10 < img.shape[1] else img.shape[1]
                    y2 = bbox[3] + 10 if bbox[3]  + 10 < img.shape[0] else img.shape[0]
                    bbox = [x1, y1, x2, y2]

                    y = 0

                    # Crop frame to bounding box
                    cropped_img = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]
                    if(cropped_img.size==0):
                        continue

                    # Resize to 100x100
                    cropped_img = cv2.resize(cropped_img,(100,100))

                    # Place cropped image onto rolling data
                    if ids[i] in list(rolling_data.keys()):
                        if len(rolling_data[ids[i]]) < 16: # bboxes values for 16 frames
                           rolling_data[ids[i]].append(np.asarray(cropped_img)) # append the image
                        else:
                           del rolling_data[ids[i]][0] # delete oldest frame bbox and append latest frame bbox
                           rolling_data[ids[i]].append(np.asarray(cropped_img))
                    else:
                      rolling_data[ids[i]] = [np.asarray(cropped_img)]

                    # Extract 16 consecutive 100x100 images for the pedestrian from the rolling data
                    if len(rolling_data[ids[i]]) == 16:
                      seq = np.stack(np.array(rolling_data[ids[i]]),axis=2) # (100*100*16*3)
                      seq = np.expand_dims(seq, axis=0)
                      y = pred_func(seq) # classification output
                    else:
                      seq = np.stack(np.array([rolling_data[ids[i]][-1]] * 16),axis=2)
                      seq = np.expand_dims(seq, axis=0)
                      y = pred_func(seq)

                    # Draw bounding box and ID label (color is based on prediction)
                    if y == 1:
                      color = (0, 0, 255) # red if crossing
                    else:
                      color = (0, 255, 0) # green if not crossing
                    img = cv2.rectangle(img, (bbox[0],bbox[1]), (bbox[2],bbox[3]), color, 2)
                    img = cv2.putText(img, str(ids[i]), org = (bbox[0],bbox[1]-5) , fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=color, thickness=2)
        else:
          img = YOLOPdetect(img)

        frame+=1


        # Set the time for this frame to the current time.
        time2 = time.time()

        # Check if the difference between the previous and this frame time > 0 to avoid division by zero.
        if (time2 - time1) > 0:

            # Calculate the number of frames per second.
            frames_per_second = 1.0 / (time2 - time1)

            # Write the calculated number of frames per second on the frame.
            cv2.putText(img, 'FPS: {}'.format(int(frames_per_second)), (10, 30),cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)

        # Update the previous frame time to this frame time.
        # As this frame will become previous frame in next iteration.
        time1 = time2

        img = cv2.putText(img, "Frame No: {}".format(frame), (120, 20),cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 0), 2)

        # cv2.imshow('BoxMOT detection', img)

        img_array = img_array+[img]
        key = cv2.waitKey(1) & 0xFF
        if key == ord(' ') or key == ord('q'):
            break

    vid.release()

    # Write output video into google drive folder
    output_video = cv2.VideoWriter(f"/content/drive/My Drive/pid_model_output/{video_name}", fourcc=cv2.VideoWriter_fourcc(*'MP4V'), fps=30,frameSize=(w,h))
    for img_ in img_array:
        output_video.write(img_)

    output_video.release()

[2024-05-10 16:43:27] INFO - processing.py - Skipping processing `KeypointsLongestMaxSizeRescale` because it resizes the image.
[2024-05-10 16:43:27] INFO - processing.py - Skipping processing `KeypointsBottomRightPadding` because it resizes the image.
[2024-05-10 16:43:27] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-05-10 16:43:28] INFO - processing.py - Skipping processing `KeypointsLongestMaxSizeRescale` because it resizes the image.
[2024-05-10 16:43:28] INFO - processing.py - Skipping processing `KeypointsBottomRightPadding` because it resizes the image.
[2024-05-10 16:43:28] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


Done. (0.156s)
inf : (0.0264s/frame)   nms : (0.0013s/frame)


[2024-05-10 16:43:28] INFO - processing.py - Skipping processing `KeypointsLongestMaxSizeRescale` because it resizes the image.
[2024-05-10 16:43:28] INFO - processing.py - Skipping processing `KeypointsBottomRightPadding` because it resizes the image.
[2024-05-10 16:43:28] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


Done. (0.155s)
inf : (0.0263s/frame)   nms : (0.0013s/frame)


[2024-05-10 16:43:28] INFO - processing.py - Skipping processing `KeypointsLongestMaxSizeRescale` because it resizes the image.
[2024-05-10 16:43:28] INFO - processing.py - Skipping processing `KeypointsBottomRightPadding` because it resizes the image.
[2024-05-10 16:43:29] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


...

Done. (0.141s)
inf : (0.0259s/frame)   nms : (0.0013s/frame)


[2024-05-10 16:48:20] INFO - processing.py - Skipping processing `KeypointsLongestMaxSizeRescale` because it resizes the image.
[2024-05-10 16:48:20] INFO - processing.py - Skipping processing `KeypointsBottomRightPadding` because it resizes the image.
[2024-05-10 16:48:20] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


Done. (0.140s)
inf : (0.0255s/frame)   nms : (0.0013s/frame)


[2024-05-10 16:48:20] INFO - processing.py - Skipping processing `KeypointsLongestMaxSizeRescale` because it resizes the image.
[2024-05-10 16:48:20] INFO - processing.py - Skipping processing `KeypointsBottomRightPadding` because it resizes the image.
[2024-05-10 16:48:20] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


Done. (0.139s)
inf : (0.0256s/frame)   nms : (0.0012s/frame)
Done. (0.142s)
inf : (0.0268s/frame)   nms : (0.0013s/frame)
